<a href="https://colab.research.google.com/github/brendenwest/ad450/blob/master/9_tree_based_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Tree-based Models

### Reading
- https://learning.oreilly.com/library/view/hands-on-machine-learning/9781491962282/ch06.html#trees_chapter
- https://learning.oreilly.com/library/view/hands-on-machine-learning/9781491962282/ch07.html#ensembles_chapter

### Tutorials
- https://www.datacamp.com/community/tutorials/decision-tree-classification-python
- https://www.datacamp.com/community/tutorials/kaggle-tutorial-machine-learning (Decision Tree Classifiers)
https://www.datacamp.com/community/tutorials/random-forests-classifier-python


### Learning Outcomes
- Decision tree models
- Measuring entropy & information gain
- Using decision trees for regression
- Ensemble learning
- Random forests
